In [10]:
import sys,os
import torch
import pytorch_lightning as pl

sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname(os.path.abspath(os.path.dirname(os.getcwd()))))))
from models.dVAE.models.model import DiscreteVAE
from models.dVAE.training.lightning import DVAETrainModule

In [32]:
tensor = torch.zeros(1, 1, 64, 64)

dvae = DiscreteVAE(
    num_tokens=2048,
    codebook_dim=256,
    num_layers=1,
    num_resnet_blocks=2,
    hidden_dim=128,
    channels=1,
    smooth_l1_loss=True,
    temperature=0.2
)

model = DVAETrainModule.load_from_checkpoint('/workspace/Haea_dev/models/dVAE/vqvae_logs/geopotential/models/geopotential-epoch=01-val_loss=0.00.ckpt')




TypeError: DVAETrainModule.__init__() missing 1 required keyword-only argument: 'dvae'

In [18]:
dvae.load_state_dict(checkpoint['state_dict'])

RuntimeError: Error(s) in loading state_dict for DiscreteVAE:
	Missing key(s) in state_dict: "codebook.weight", "encoder.0.0.weight", "encoder.0.0.bias", "encoder.1.net.0.weight", "encoder.1.net.0.bias", "encoder.1.net.2.weight", "encoder.1.net.2.bias", "encoder.1.net.4.weight", "encoder.1.net.4.bias", "encoder.2.net.0.weight", "encoder.2.net.0.bias", "encoder.2.net.2.weight", "encoder.2.net.2.bias", "encoder.2.net.4.weight", "encoder.2.net.4.bias", "encoder.3.weight", "encoder.3.bias", "decoder.0.weight", "decoder.0.bias", "decoder.1.net.0.weight", "decoder.1.net.0.bias", "decoder.1.net.2.weight", "decoder.1.net.2.bias", "decoder.1.net.4.weight", "decoder.1.net.4.bias", "decoder.2.net.0.weight", "decoder.2.net.0.bias", "decoder.2.net.2.weight", "decoder.2.net.2.bias", "decoder.2.net.4.weight", "decoder.2.net.4.bias", "decoder.3.0.weight", "decoder.3.0.bias", "decoder.4.weight", "decoder.4.bias". 
	Unexpected key(s) in state_dict: "dvae.codebook.weight", "dvae.encoder.0.0.weight", "dva